In [ ]:
import os
from pathlib import Path
import pandas as pd
from IPython.display import display


pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10
pd.set_option("display.max_colwidth", None)


targets = [
    "United States",
    "United Kingdom",
    "Canada",
    "Australia",
    "Germany",
    "Japan",
    "Netherlands",
]


def read_data(path: Path) -> pd.DataFrame:
    return pd.read_json(path)


for i in Path(os.getenv("DATA_DIR")).rglob("*.json"):
    print(i)
    df = read_data(i)
    df = df[df["country"].isin(targets)]
    if "year" in df.columns:
        df = df[df["year"].between(2010, 2024)]
    display(df)

/Users/yo/Developer/repo/sign-to-migrate/data/cpi.json


,countryiso3code,country,year,value
89,AUS,Australia,2023,139.880333
90,AUS,Australia,2022,132.466181
91,AUS,Australia,2021,124.271592
92,AUS,Australia,2020,120.811655
397,CAN,Canada,2023,134.895535
...,...,...,...,...
2393,GBR,United Kingdom,2020,120.806362
2404,USA,United States,2023,139.735794
2405,USA,United States,2022,134.211206
2406,USA,United States,2021,124.266414


/Users/yo/Developer/repo/sign-to-migrate/data/unemployment_rate.json


,country,source,indicator,sex,classif1,year,obs_value,obs_status,note_classif,note_indicator,note_source
768,Australia,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15+",2023,3.668,None,None,None,Repository: ILO-STATISTICS - Micro data processing
769,Australia,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15-24",2023,8.319,None,None,None,Repository: ILO-STATISTICS - Micro data processing
770,Australia,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 25+",2023,2.774,None,None,None,Repository: ILO-STATISTICS - Micro data processing
771,Australia,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15+",2023,3.755,None,None,None,Repository: ILO-STATISTICS - Micro data processing
772,Australia,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15-24",2023,9.175,None,None,None,Repository: ILO-STATISTICS - Micro data processing
...,...,...,...,...,...,...,...,...,...,...,...
15852,Netherlands,LFS - EU Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15-24",2020,9.197,None,None,None,Repository: ILO-STATISTICS - Micro data processing
15853,Netherlands,LFS - EU Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 25+",2020,2.751,None,None,None,Repository: ILO-STATISTICS - Micro data processing
15854,Netherlands,LFS - EU Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15+",2020,3.953,None,None,None,Repository: ILO-STATISTICS - Micro data processing
15855,Netherlands,LFS - EU Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15-24",2020,9.015,None,None,None,Repository: ILO-STATISTICS - Micro data processing


/Users/yo/Developer/repo/sign-to-migrate/data/women_in_senior_and_middle_position.json


...

,country,source,indicator,sex,classif1,year,obs_value,obs_status,note_classif,note_indicator,note_source
1603,Germany,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Total,2022,5.803,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
1604,Germany,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Persons with disability,2022,32.238,None,Nonstandard definition of disability: Based on limitation in activities because of health problems,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
1605,Germany,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Persons without disability,2022,5.287,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
1606,Germany,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Total,2022,6.328,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
1607,Germany,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons with disability,2022,37.900,None,Nonstandard definition of disability: Based on limitation in activities because of health problems,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
...,...,...,...,...,...,...,...,...,...,...,...
5373,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons with disability,2020,20.016,Unreliable,Nonstandard definition of disability: Based on limitation in activities because of health problems,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
5374,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons without disability,2020,3.470,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
5375,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Female,Disability status (Aggregate): Total,2020,3.895,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
5376,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Female,Disability status (Aggregate): Persons with disability,2020,NaN,Unreliable,Nonstandard definition of disability: Based on limitation in activities because of health problems,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old


/Users/yo/Developer/repo/sign-to-migrate/data/informal_employment.json


,country,source,indicator,sex,classif1,year,obs_value,obs_status,note_classif,note_indicator,note_source
1471,Australia,"HIES - Household, Income and Labour Dynamics Survey",SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Total",2021,26.105,None,None,None,Repository: ILO-STATISTICS - Micro data processing
1472,Australia,"HIES - Household, Income and Labour Dynamics Survey",SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Agriculture",2021,63.608,None,None,None,Repository: ILO-STATISTICS - Micro data processing
1473,Australia,"HIES - Household, Income and Labour Dynamics Survey",SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Non-agriculture",2021,25.214,None,None,None,Repository: ILO-STATISTICS - Micro data processing
1474,Australia,"HIES - Household, Income and Labour Dynamics Survey",SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Male,"Economic activity (Agriculture, Non-Agriculture): Total",2021,25.753,None,None,None,Repository: ILO-STATISTICS - Micro data processing
1475,Australia,"HIES - Household, Income and Labour Dynamics Survey",SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Male,"Economic activity (Agriculture, Non-Agriculture): Agriculture",2021,59.271,None,None,None,Repository: ILO-STATISTICS - Micro data processing
...,...,...,...,...,...,...,...,...,...,...,...
35335,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,Economic activity (Aggregate): Agriculture,2020,NaN,Unreliable,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
35336,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,Economic activity (Aggregate): Manufacturing,2020,NaN,Unreliable,Nonstandard economic activity: Including B | Nonstandard economic activity: Including D-E,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
35337,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,"Economic activity (Aggregate): Trade, Transportation, Accommodation and Food, and Business and Administrative Services",2020,1.360,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old
35338,Netherlands,HIES - EU Statistics on Income and Living Conditions,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,"Economic activity (Aggregate): Public Administration, Community, Social and other Services and Activities",2020,3.673,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Age coverage - minimum age: 16 years old


/Users/yo/Developer/repo/sign-to-migrate/data/happiness.json


,Overall rank,country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,year,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Dystopia Residual,Lower Confidence Interval,Upper Confidence Interval,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Trust..Government.Corruption.,Dystopia.Residual
